# Load packages

In [1]:
import numpy as np
import pandas as pd

# Combine all tournament and reg season games

In [2]:
tourney_compact_df = pd.read_csv('Data/Kaggle NCAA/NCAATourneyCompactResults.csv')
tourney_compact_df['is_tourney'] = 1.0

In [3]:
regseason_compact_df = pd.read_csv('Data/Kaggle NCAA/RegularSeasonCompactResults.csv')
regseason_compact_df['is_tourney'] = 0.0

In [4]:
all_games_df = regseason_compact_df.append(tourney_compact_df)
all_games_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,is_tourney
0,1985,20,1228,81,1328,64,N,0,0.0
1,1985,25,1106,77,1354,70,H,0,0.0
2,1985,25,1112,63,1223,56,H,0,0.0
3,1985,25,1165,70,1432,54,H,0,0.0
4,1985,25,1192,86,1447,74,H,0,0.0


In [5]:
all_games_df.tail()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,is_tourney
2179,2018,146,1242,85,1181,81,N,1,1.0
2180,2018,146,1437,71,1403,59,N,0,1.0
2181,2018,152,1276,69,1260,57,N,0,1.0
2182,2018,152,1437,95,1242,79,N,0,1.0
2183,2018,154,1437,79,1276,62,N,0,1.0


# Add spread, seeds, ranks, and Ken POM data

Load and merge on point spread

In [6]:
spread_df = pd.read_csv('Data/~Created data/spreads_all.csv')

C:\Users\dlorenz\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3,6,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
spread_df = spread_df[['Season','date','line','lineavg','road','home','rscore','hscore','WScore','LScore','HTeamID','RTeamID','WTeamID','LTeamID']]

Merge point spread on to games

In [8]:
all_games_df = pd.merge(all_games_df, spread_df, left_on=['WTeamID','LTeamID','Season','WScore','LScore'], right_on = ['WTeamID','LTeamID','Season','WScore','LScore'], how='inner')

C:\Users\dlorenz\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:946: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  'representation', UserWarning)


In [9]:
all_games_df = all_games_df[['Season','date','is_tourney','home','HTeamID','hscore','road','RTeamID','rscore','line','lineavg']]

In [10]:
all_games_df['Hwin'] = 0
all_games_df.loc[all_games_df['hscore']>all_games_df['rscore'], 'Hwin'] = 1      

In [11]:
all_games_df.head()

,Season,date,is_tourney,home,HTeamID,hscore,road,RTeamID,rscore,line,lineavg,Hwin
0,2004,11/24/2003,0.0,arizona st.,1113.0,83,cal st. fullerton,1168.0,76,19.0,23.20,1
1,2004,11/24/2003,0.0,dayton,1173.0,82,central michigan,1141.0,63,11.0,5.88,1
2,2004,11/24/2003,0.0,detroit,1178.0,64,western michigan,1444.0,43,1.0,5.63,1
3,2004,11/24/2003,0.0,st. john's,1385.0,59,fairfield,1193.0,64,12.5,13.81,0
4,2004,11/24/2003,0.0,gonzaga,1211.0,90,denver,1176.0,58,19.5,18.45,1


Add seeds

In [12]:
seeds_df = pd.read_csv('Data/Kaggle NCAA/NCAATourneySeeds.csv')
seeds_df['Seed_num'] = seeds_df['Seed'].str.extract('(\d\d)', expand=True)
seeds_df['Seed_num'] = pd.to_numeric(seeds_df['Seed_num'])
seeds_df = seeds_df[['Season','TeamID','Seed_num']]
seeds_df.rename(columns={'Seed_num': 'Seed'}, inplace=True)
seeds_df.head()

,Season,TeamID,Seed
0,1985,1207,1
1,1985,1210,2
2,1985,1228,3
3,1985,1260,4
4,1985,1374,5


In [13]:
#Note: we merge twice for this merge and subsequent merges, because the data are in a wide format by team (i.e., there are two variables rather than two rows for each game)
home_road = ['H','R']
for hr in home_road:
    all_games_df = pd.merge(all_games_df, seeds_df, left_on=[hr+'TeamID','Season'], right_on = ['TeamID','Season'], how='left')
    all_games_df.rename(columns={'Seed': hr+'Seed'}, inplace=True)
    all_games_df = all_games_df.drop(['TeamID'], axis=1)

In [14]:
all_games_df.head()

,Season,date,is_tourney,home,HTeamID,hscore,road,RTeamID,rscore,line,lineavg,Hwin,HSeed,RSeed
0,2004,11/24/2003,0.0,arizona st.,1113.0,83,cal st. fullerton,1168.0,76,19.0,23.20,1,NaN,NaN
1,2004,11/24/2003,0.0,dayton,1173.0,82,central michigan,1141.0,63,11.0,5.88,1,10.0,NaN
2,2004,11/24/2003,0.0,detroit,1178.0,64,western michigan,1444.0,43,1.0,5.63,1,NaN,11.0
3,2004,11/24/2003,0.0,st. john's,1385.0,59,fairfield,1193.0,64,12.5,13.81,0,NaN,NaN
4,2004,11/24/2003,0.0,gonzaga,1211.0,90,denver,1176.0,58,19.5,18.45,1,2.0,NaN


Add Ken Pom data

In [15]:
kp_df = pd.read_csv('Data/~Created data/kp_all.csv')

In [16]:
efficiency_list = ['conf','adjem','adjo','adjd','luck']
for hr in home_road:
    all_games_df = pd.merge(all_games_df, kp_df, left_on=[hr+'TeamID','Season'], right_on = ['TeamID','Season'], how='inner')
    for metric in efficiency_list:
        all_games_df.rename(columns={metric: hr+metric}, inplace=True)
    all_games_df = all_games_df.drop(['TeamID','team'], axis=1)

In [17]:
all_games_df.head()

,Season,date,is_tourney,home,HTeamID,hscore,road,RTeamID,rscore,line,...,Hconf,Hadjem,Hadjo,Hadjd,Hluck,Rconf,Radjem,Radjo,Radjd,Rluck
0,2004,11/24/2003,0.0,arizona st.,1113.0,83,cal st. fullerton,1168.0,76,19.0,...,P10,1.29,107.6,106.3,-0.049,BW,-4.85,102.9,107.8,-0.023
1,2004,12/01/2003,0.0,pepperdine,1337.0,74,cal st. fullerton,1168.0,78,17.0,...,WCC,2.93,113.7,110.8,-0.001,BW,-4.85,102.9,107.8,-0.023
2,2004,01/17/2004,0.0,uc irvine,1414.0,74,cal st. fullerton,1168.0,67,7.0,...,BW,-4.79,101.4,106.2,0.034,BW,-4.85,102.9,107.8,-0.023
3,2004,01/29/2004,0.0,cal poly slo,1142.0,80,cal st. fullerton,1168.0,71,5.5,...,BW,-4.69,101.1,105.8,-0.015,BW,-4.85,102.9,107.8,-0.023
4,2004,02/21/2004,0.0,utah st.,1429.0,87,cal st. fullerton,1168.0,61,17.0,...,BW,13.60,115.9,102.3,0.037,BW,-4.85,102.9,107.8,-0.023


Add Massey and Ken Pom rankings

In [18]:
massey_df = pd.read_csv('Data/Kaggle NCAA/MasseyOrdinals_thru_2019_day_128.csv')
massey_df19 = massey_df.loc[massey_df['Season'] == 2019]

#Ranking at the end of season versus end of tourney (as shown in KP data)
POM_df_else = massey_df[massey_df['SystemName'].str.contains("POM")]
POM_end_df_else = POM_df_else.loc[POM_df_else['RankingDayNum'] == 133]
POM_df19 = massey_df19[massey_df19['SystemName'].str.contains("POM")]
POM_end_df_19 = POM_df19.loc[POM_df19['RankingDayNum'] == 128]
POM_end_df = POM_end_df_else.append(POM_end_df_19)
POM_end_df = POM_end_df[['Season','TeamID','OrdinalRank']]
POM_end_df.rename(columns={'OrdinalRank': 'RankPOM'}, inplace=True)

In [19]:
for hr in home_road:
    all_games_df = pd.merge(all_games_df,POM_end_df, left_on=[hr+'TeamID','Season'], right_on = ['TeamID','Season'], how='left')
    all_games_df.rename(columns={'RankPOM': hr+'RankPOM'}, inplace=True)
    all_games_df = all_games_df.drop(['TeamID'], axis=1)

# Calculate regular season avg stats

In [20]:
regseason_detail_df = pd.read_csv('Data/Kaggle NCAA/RegularSeasonDetailedResults.csv')

In [21]:
regseason_detail_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [22]:
#Attach the POM data
teamslist = ['W','L']
for wl in teamslist:
    regseason_detail_df = pd.merge(regseason_detail_df, POM_end_df, left_on=[wl+'TeamID','Season'], right_on = ['TeamID','Season'], how='left')
    regseason_detail_df.rename(columns={'RankPOM': wl+'RankPOM'}, inplace=True)
    regseason_detail_df = regseason_detail_df.drop(['TeamID'], axis=1)

Regular season (create one row for each team with avg stats and win info)

In [23]:
win_team_only = regseason_detail_df.drop(['LTeamID','WAst','LAst','LStl','WStl','LBlk','WBlk','LPF','WPF','NumOT','DayNum'], axis=1)
win_team_only.rename(columns={'WTeamName': 'TeamName'}, inplace=True)
win_team_only.rename(columns={'WTeamID': 'TeamID'}, inplace=True)

team_factors_list = ['Score','FGM','FGA','FGA3','FGM3','TO','FTM','FTA','RankPOM']
for factor in team_factors_list:
    win_team_only.rename(columns={'W'+factor: factor}, inplace=True)
    win_team_only.rename(columns={'L'+factor: 'Opponent'+factor}, inplace=True)

win_team_only['wins'] = 1 

win_team_only['wins_top25'] = 0
win_team_only.loc[win_team_only['OpponentRankPOM']<26, 'wins_top25'] = 1

win_team_only['wins_top5'] = 0
win_team_only.loc[win_team_only['OpponentRankPOM']<6, 'wins_top5'] = 1

win_team_only = win_team_only[['Season','TeamID','wins','wins_top25','wins_top5','Score','OpponentScore','FGM','FGA','OpponentFGM','OpponentFGA','FGM3','FGA3','OpponentFGM3','OpponentFGA3','FTM','FTA','OpponentFTM','OpponentFTA','TO','OpponentTO']]

In [24]:
loss_team_only = regseason_detail_df.drop(['WTeamID','WAst','LAst','LStl','WStl','LBlk','WBlk','LPF','WPF','NumOT','DayNum'], axis=1)
loss_team_only.rename(columns={'LTeamName': 'TeamName'}, inplace=True)
loss_team_only.rename(columns={'LTeamID': 'TeamID'}, inplace=True)

for factor in team_factors_list:
    loss_team_only.rename(columns={'L'+factor: factor}, inplace=True)
    loss_team_only.rename(columns={'W'+factor: 'Opponent'+factor}, inplace=True)

loss_team_only['wins'] = 0
loss_team_only['wins_top25'] = 0
loss_team_only['wins_top5'] = 0

loss_team_only = loss_team_only[['Season','TeamID','wins','wins_top25','wins_top5','Score','OpponentScore','FGM','FGA','OpponentFGM','OpponentFGA','FGM3','FGA3','OpponentFGM3','OpponentFGA3','FTM','FTA','OpponentFTM','OpponentFTA','TO','OpponentTO']]

In [25]:
reg_season_all = win_team_only.append(loss_team_only)

In [26]:
reg_season_all['TOmargin'] = reg_season_all['TO']-reg_season_all['OpponentTO']

reg_season_all['PointMargin'] = reg_season_all['Score']-reg_season_all['OpponentScore']

reg_season_all['FG'] = reg_season_all['FGM']/reg_season_all['FGA']
reg_season_all['FGopponent'] = reg_season_all['OpponentFGM']/reg_season_all['OpponentFGA']

reg_season_all['FG3'] = reg_season_all['FGM3']/reg_season_all['FGA3']
reg_season_all['FG3opponent'] = reg_season_all['OpponentFGM3']/reg_season_all['OpponentFGA3']

reg_season_all['FT'] = reg_season_all['FTM']/reg_season_all['FTA']

In [27]:
reg_season_all = reg_season_all.drop(['TO','OpponentTO','FGM','FGA','OpponentFGA','OpponentFGM','FGM3','FGA3','OpponentFGM3','OpponentFGA3','FTM','FTA','OpponentFTM','OpponentFTA'], axis=1)

In [28]:
list(reg_season_all)

['Season',
 'TeamID',
 'wins',
 'wins_top25',
 'wins_top5',
 'Score',
 'OpponentScore',
 'TOmargin',
 'PointMargin',
 'FG',
 'FGopponent',
 'FG3',
 'FG3opponent',
 'FT']

# Collapse regular season sums/means

In [29]:
reg_season_means = reg_season_all.groupby(['TeamID','Season']).mean().reset_index()

In [30]:
reg_season_means = reg_season_means.drop(['wins','wins_top25','wins_top5'], axis=1)

In [31]:
reg_season_sum = reg_season_all.groupby(['TeamID','Season']).sum().reset_index()

In [32]:
reg_season_sum = reg_season_sum.drop(['Score','OpponentScore','TOmargin','PointMargin','FG','FG3','FGopponent','FG3opponent','FT'], axis=1)

In [33]:
regseason_df = pd.merge(reg_season_means, reg_season_sum, left_on=['TeamID','Season'], right_on = ['TeamID','Season'], how='left')

In [34]:
regseason_df.head()

,TeamID,Season,Score,OpponentScore,TOmargin,PointMargin,FG,FGopponent,FG3,FG3opponent,FT,wins,wins_top25,wins_top5
0,1101,2014,63.142857,78.619048,2.857143,-15.476190,0.410184,0.510776,0.380275,0.369975,0.758031,2,0,0
1,1101,2015,61.000000,71.857143,-0.642857,-10.857143,0.407501,0.493864,0.381507,0.372684,0.736328,7,0,0
2,1101,2016,69.851852,76.259259,-0.037037,-6.407407,0.442606,0.493308,0.364517,0.366441,0.701402,9,0,0
3,1101,2017,67.880000,72.640000,-0.320000,-4.760000,0.457867,0.463728,0.371026,0.354196,0.640438,9,0,0
4,1101,2018,70.666667,72.814815,-1.074074,-2.148148,0.452366,0.458319,0.327240,0.316082,0.705605,12,0,0


In [35]:
regseason_df.tail()

,TeamID,Season,Score,OpponentScore,TOmargin,PointMargin,FG,FGopponent,FG3,FG3opponent,FT,wins,wins_top25,wins_top5
5829,1464,2017,76.000000,83.187500,-0.312500,-7.187500,0.435333,0.470456,0.325612,0.346281,0.676003,11,0,0
5830,1464,2018,72.833333,83.933333,-0.033333,-11.100000,0.420161,0.516583,0.331173,0.404887,0.648439,6,0,0
5831,1464,2019,73.500000,79.233333,1.633333,-5.733333,0.422696,0.470398,0.348279,0.338438,0.692387,10,0,0
5832,1465,2019,75.461538,75.192308,1.692308,0.269231,0.444370,0.436562,0.351052,0.355434,0.770982,12,0,0
5833,1466,2019,65.620690,74.931034,1.482759,-9.310345,0.384144,0.461113,0.295101,0.337730,0.675597,7,0,0


In [36]:
regseason_df.to_csv('Data/~Created data/regseason_df.csv', index=False)

# Combine regular season stats with game level data

In [37]:
#Merge the avg regular season performance with all games and have performance wide: 
performance_list = ['wins','wins_top5','wins_top25','Score','OpponentScore','TOmargin','PointMargin','FG','FGopponent','FG3','FG3opponent','FT','RBMargin']
for hr in home_road:
    all_games_df = pd.merge(all_games_df, regseason_df, left_on=[hr+'TeamID','Season'], right_on = ['TeamID','Season'], how='left')
    for var in performance_list:
        all_games_df.rename(columns={var: hr+var}, inplace=True)
    all_games_df = all_games_df.drop(['TeamID'], axis=1)

# Add binary experience dummy

In [38]:
temp_df = all_games_df[all_games_df['is_tourney'] == 1]
temp_df_home = temp_df[['home']]
temp_df_home.rename(columns={'home': 'team'}, inplace=True)
temp_df_road = temp_df[['road']]
temp_df_road.rename(columns={'road': 'team'}, inplace=True)
tourney_teams_df = temp_df_home.append(temp_df_road)
tourney_teams_df['team'].value_counts(dropna=False)

C:\Users\dlorenz\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


kansas               49
north carolina       47
kentucky             44
duke                 43
michigan st.         42
wisconsin            39
florida              39
villanova            37
gonzaga              36
louisville           34
arizona              32
connecticut          31
xavier               31
syracuse             31
butler               27
ohio st.             26
ucla                 26
west virginia        26
texas                24
michigan             23
pittsburgh           23
memphis              23
oregon               21
wichita st.          19
marquette            18
oklahoma             18
cincinnati           18
purdue               18
tennessee            18
notre dame           18
                     ..
nc greensboro         1
uc irvine             1
texas san antonio     1
north florida         1
delaware              1
indiana st.           1
penn st.              1
miami-ohio            1
loyola-maryland       1
lamar                 1
jacksonville st.

In [39]:
all_games_df['Htourny20plus'] = 0
all_games_df['Rtourny20plus'] = 0

experienced_teams = ['kansas','north carolina','kentucky','duke','michigan st.','wisconsin','florida','villanova','gonzaga','louisville','arizona','xavier','connecticut','syracuse','butler','ohio st.','ucla','west virginia','texas','michigan','pittsburgh','memphis','oregon']
for team in experienced_teams:
    all_games_df.loc[all_games_df['home']==team, 'Htourny20plus'] = 1
    all_games_df.loc[all_games_df['road']==team, 'Rtourny20plus'] = 1

In [40]:
temp_df = all_games_df[all_games_df['is_tourney'] == 1]
temp_df_home = temp_df[['Hconf']]
temp_df_home.rename(columns={'Hconf': 'conf'}, inplace=True)
temp_df_road = temp_df[['Rconf']]
temp_df_road.rename(columns={'Rconf': 'conf'}, inplace=True)
tourny_teams_df = temp_df_home.append(temp_df_road)
tourny_teams_df['conf'].value_counts(dropna=False)

C:\Users\dlorenz\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


BE      229
ACC     217
B10     203
B12     190
SEC     170
A10      85
P10      73
P12      66
MWC      56
CUSA     52
WCC      51
MVC      51
CAA      32
Horz     31
Amer     28
WAC      23
BW       21
MAC      19
OVC      19
MAAC     19
Ivy      19
SC       18
ASun     18
Pat      18
AE       16
BSky     16
MEAC     16
BSth     15
SB       14
NEC      14
Slnd     14
Sum      12
SWAC     10
MCon      3
Name: conf, dtype: int64

In [41]:
all_games_df['HBig4Conf'] = 0
all_games_df['RBig4Conf'] = 0
conferences = ['ACC','B10','B12','SEC']
for conf in conferences:
    all_games_df.loc[all_games_df['Hconf']==conf, 'HBig4Conf'] = 1
    all_games_df.loc[all_games_df['Rconf']==conf, 'RBig4Conf'] = 1

In [42]:
all_games_df.tail()

,Season,date,is_tourney,home,HTeamID,hscore,road,RTeamID,rscore,line,...,RFG3,RFG3opponent,RFT,Rwins,Rwins_top25,Rwins_top5,Htourny20plus,Rtourny20plus,HBig4Conf,RBig4Conf
54057,2019,2/9/2019,0.0,central arkansas,1146.0,67,se louisiana,1368.0,75,2.5,...,0.334396,0.333529,0.70402,15,0,0,0,0,0,0
54058,2019,2/20/2019,0.0,new orleans,1309.0,89,se louisiana,1368.0,68,2.5,...,0.334396,0.333529,0.70402,15,0,0,0,0,0,0
54059,2019,1/19/2019,0.0,mcneese st.,1270.0,71,se louisiana,1368.0,74,3.5,...,0.334396,0.333529,0.70402,15,0,0,0,0,0,0
54060,2019,12/19/2018,0.0,grambling st.,1212.0,69,se louisiana,1368.0,68,6.0,...,0.334396,0.333529,0.70402,15,0,0,0,0,0,0
54061,2019,3/9/2019,0.0,nicholls st.,1311.0,69,se louisiana,1368.0,63,-1.5,...,0.334396,0.333529,0.70402,15,0,0,0,0,0,0


In [43]:
list(all_games_df)

['Season',
 'date',
 'is_tourney',
 'home',
 'HTeamID',
 'hscore',
 'road',
 'RTeamID',
 'rscore',
 'line',
 'lineavg',
 'Hwin',
 'HSeed',
 'RSeed',
 'Hconf',
 'Hadjem',
 'Hadjo',
 'Hadjd',
 'Hluck',
 'Rconf',
 'Radjem',
 'Radjo',
 'Radjd',
 'Rluck',
 'HRankPOM',
 'RRankPOM',
 'HScore',
 'HOpponentScore',
 'HTOmargin',
 'HPointMargin',
 'HFG',
 'HFGopponent',
 'HFG3',
 'HFG3opponent',
 'HFT',
 'Hwins',
 'Hwins_top25',
 'Hwins_top5',
 'RScore',
 'ROpponentScore',
 'RTOmargin',
 'RPointMargin',
 'RFG',
 'RFGopponent',
 'RFG3',
 'RFG3opponent',
 'RFT',
 'Rwins',
 'Rwins_top25',
 'Rwins_top5',
 'Htourny20plus',
 'Rtourny20plus',
 'HBig4Conf',
 'RBig4Conf']

Generate share differences between teams

In [44]:
#Note: only fills in seeds and some games missing line
#mean imputations of variables for model
all_games_df = all_games_df.fillna(all_games_df.mean())

# Output to csv

In [45]:
all_games_df.to_csv('Data/~Created data/all_games_df.csv', index=False)